Ниже выписаны мои наблюдения для твитов. Ниже произведена попытка их имплементировать.
* повы вклад дает +
* повы ипотек дает -
* повы кредит дает -
* сни ипотек дает +
* сни кредит дает +
* санкц дает -
* города дают 0
* конструкция "как (можно/легко) (взять/узнать/получить/оформить)" дает 0
* http://... - idf высокий. значит надо исключить слова с оригинальной частью ссылки. можно оставить http, оно указывает на 0
* пунктуация, ники пользователей, стоп слова и цифры дают достаточно высокий idf, их исключаем, они не оказывают влияние на тональность

План разбивается на следующие этапы:
1.   Загрузить корпус
2.   Во время загрузки избавиться от ссылок, пунктуации, чисел и ников + стоп-слов
3. Для устранения омонимии провести лемматизацию с помощью spacy
4. Написать функцию, определяющую наличие городов в твите (куда вставить?)
5. Проверить точность выделения признаков с помощью метрики Хи-квадрат





---



---



In [104]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [105]:
from lxml import etree

In [106]:
from typing import List, Tuple

In [107]:
import string, re

In [108]:
%pip install spacy-udpipe

  Using cached pydantic-1.8.2-cp37-cp37m-manylinux2014_x86_64.whl (10.1 MB)
  Using cached typer-0.4.0-py3-none-any.whl (27 kB)
  Attempting uninstall: typer
    Found existing installation: typer 0.2.1
    Uninstalling typer-0.2.1:
      Successfully uninstalled typer-0.2.1
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.4
    Uninstalling pydantic-1.4:
      Successfully uninstalled pydantic-1.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
toponym 0.5.1 requires pydantic==1.4, but you have pydantic 1.8.2 which is incompatible.
toponym 0.5.1 requires typer==0.2.1, but you have typer 0.4.0 which is incompatible.
flashgeotext 0.4.0 requires loguru<0.6.0,>=0.5.3, but you have loguru 0.4.1 which is incompatible.


In [109]:
import spacy_udpipe

In [110]:
%pip install pymorphy2==0.8

In [111]:
spacy_udpipe.download("ru-syntagrus") 
nlp = spacy_udpipe.load("ru-syntagrus")

Already downloaded a model for the 'ru-syntagrus' language




---
Функция для нахождения названий городов


In [112]:
%pip install flashgeotext

  Using cached loguru-0.5.3-py3-none-any.whl (57 kB)
  Attempting uninstall: loguru
    Found existing installation: loguru 0.4.1
    Uninstalling loguru-0.4.1:
      Successfully uninstalled loguru-0.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
toponym 0.5.1 requires loguru==0.4.1, but you have loguru 0.5.3 which is incompatible.
toponym 0.5.1 requires pydantic==1.4, but you have pydantic 1.8.2 which is incompatible.
toponym 0.5.1 requires typer==0.2.1, but you have typer 0.4.0 which is incompatible.


In [113]:
import flashgeotext

In [114]:
import json

In [115]:
%pip install toponym

  Using cached loguru-0.4.1-py3-none-any.whl (54 kB)
  Using cached pydantic-1.4-cp37-cp37m-manylinux2010_x86_64.whl (7.5 MB)
  Using cached typer-0.2.1-py3-none-any.whl (20 kB)
  Attempting uninstall: typer
    Found existing installation: typer 0.4.0
    Uninstalling typer-0.4.0:
      Successfully uninstalled typer-0.4.0
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.8.2
    Uninstalling pydantic-1.8.2:
      Successfully uninstalled pydantic-1.8.2
  Attempting uninstall: loguru
    Found existing installation: loguru 0.5.3
    Uninstalling loguru-0.5.3:
      Successfully uninstalled loguru-0.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.0.10 requires pydantic!=1.8,!=1.8.1,<1.9.0,>=1.7.4, but you have pydantic 1.4 which is incompatible.
spacy 3.1.3 requires pydantic!=1.8,!=1.8.1,<1.9.0,>=1.7.4, but you have pydant

In [116]:
from toponym.recipes import Recipes
from toponym.toponym import Toponym
recipes = Recipes()
recipes.load_from_language(language="russian")

In [117]:
list_of_cities = []
with open('/content/drive/MyDrive/Colab Notebooks/russian-cities.json', 'r', encoding="utf8") as f:
    data = json.load(f)
    for num in range(len(data)):
      list_of_cities.append(data[num]['name'])

dict_of_cities = dict()
for city in list_of_cities:
    t = Toponym(input_word=city, recipes=recipes)
    t.build()
    t_low = Toponym(input_word=city.lower(), recipes=recipes)
    t_low.build()
    both = t_low.list_toponyms() + t.list_toponyms()
    dict_of_cities[city] = both
print(dict_of_cities)

2021-10-06 07:22:36.413 | WARNING  | toponym.recipes:__getitem__:30 - No word_ending found. Using _default
2021-10-06 07:22:36.416 | WARNING  | toponym.recipes:__getitem__:30 - No word_ending found. Using _default
2021-10-06 07:22:36.427 | WARNING  | toponym.recipes:__getitem__:30 - No word_ending found. Using _default
2021-10-06 07:22:36.429 | WARNING  | toponym.recipes:__getitem__:30 - No word_ending found. Using _default
2021-10-06 07:22:36.447 | WARNING  | toponym.recipes:__getitem__:30 - No word_ending found. Using _default
2021-10-06 07:22:36.449 | WARNING  | toponym.recipes:__getitem__:30 - No word_ending found. Using _default
2021-10-06 07:22:36.463 | WARNING  | toponym.recipes:__getitem__:30 - No word_ending found. Using _default
2021-10-06 07:22:36.465 | WARNING  | toponym.recipes:__getitem__:30 - No word_ending found. Using _default
2021-10-06 07:22:36.469 | WARNING  | toponym.recipes:__getitem__:30 - No word_ending found. Using _default
2021-10-06 07:22:36.470 | WARNING  | 

{'Абаза': ['абаза', 'абазой', 'абазе', 'абази', 'абазы', 'абазу', 'Абазой', 'Абаза', 'Абазу', 'Абази', 'Абазы', 'Абазе'], 'Абакан': ['абаканю', 'абаканя', 'абакане', 'абакана', 'абакану', 'абаканом', 'абакан', 'абаканем', 'Абакану', 'Абаканем', 'Абакане', 'Абаканом', 'Абакан', 'Абакана', 'Абаканю', 'Абаканя'], 'Абдулино': ['абдулинем', 'абдулину', 'абдулиня', 'абдулине', 'абдулино', 'абдулином', 'абдулиню', 'абдулина', 'Абдулиню', 'Абдулину', 'Абдулина', 'Абдулино', 'Абдулином', 'Абдулине', 'Абдулинем', 'Абдулиня'], 'Абинск': ['абинскя', 'абинску', 'абинском', 'абинск', 'абинскю', 'абинскем', 'абинска', 'абинске', 'Абинска', 'Абинскя', 'Абинск', 'Абинском', 'Абинску', 'Абинске', 'Абинскем', 'Абинскю'], 'Агидель': ['агидель', 'агидели', 'агиделью', 'Агидель', 'Агиделью', 'Агидели'], 'Агрыз': ['агрыз', 'агрыза', 'агрызу', 'агрызем', 'агрызом', 'агрызю', 'агрызе', 'агрызя', 'Агрызя', 'Агрыз', 'Агрыза', 'Агрызем', 'Агрызю', 'Агрызе', 'Агрызу', 'Агрызом'], 'Адыгейск': ['адыгейске', 'адыгейс

In [118]:
from flashgeotext.geotext import GeoText, GeoTextConfiguration
from flashgeotext.lookup import LookupData
example_text = "http://t.co/YEVHuvVGA1 Взять кредит тюмень альфа банк. Мнение о кредитной карте втб 24 http://t.co/SBJTcsqjCg. «Райффайзенбанк»: Снижение ключевой ставки ЦБ на заседании в Тюмени в эту пятницу очень маловероятно. Современное состояние кредитного поведения в россии сбербанк http://t.co/EXAX01uR0h. @sawik_shuster @YevhenS Главное чтоб банки СБЕР и ВТБ!!!"

lookup_cities = LookupData(
    name="Russian_cities",
    data=dict_of_cities,
    script="cyrillic")

config = GeoTextConfiguration(**{"use_demo_data":False})
geotext = GeoText(config)
geotext.add(lookup_cities)

extract = geotext.extract(example_text)
print(extract)

{'Russian_cities': {'Тюмень': {'count': 2, 'span_info': [(36, 42), (172, 178)], 'found_as': ['тюмень', 'Тюмени']}}}




---

Загрузка и очистка корпуса

In [119]:
def rubbish_deleter(s):
  lemmas = []
  for token in nlp(s):
    if token.is_stop: # проверка на стоп слово
      pass
    else:
      lemmas.append(token.lemma_) # тут слова лемматизируются с помощью spacy
  s_lemmd = ' '.join(lemmas)                                                
  webs = '((?<=http:\/\/)t\.\S+\.?\S+[\s^\v]?)'
  nums = '(\d+[\s^\v]?)'
  names = '(@\S+)'
  re_webs = re.compile(webs)                                    # Ссылки
  re_names = re.compile(names)                                  # Ники
  re_pnct = re.compile('[%s]' % re.escape(string.punctuation))  # Пунктуация
  re_nums = re.compile(nums)                                    # Числа
  s_dewebbed = re_webs.sub(' ', s_lemmd)
  s_denamed = re_names.sub('', s_dewebbed)
  s_stripped = re_pnct.sub('', s_denamed)
  s_denummed = re_nums.sub('', s_stripped)
  
  return s_denummed

In [120]:
def load_sentirueval_2016(file_name: str) -> Tuple[List[str], List[str]]:
    texts = []
    labels = []
    with open(file_name, mode='rb') as fp:
        xml_data = fp.read()
    root = etree.fromstring(xml_data)
    for database in root.getchildren():
        if database.tag == 'database':
            for table in database.getchildren():
                if table.tag != 'table':
                    continue
                new_text = None
                new_label = None
                for column in table.getchildren():
                    if column.get('name') == 'text':
                        new_text = rubbish_deleter(column.text) # дополнено функцией тут
                        if new_label is not None:
                            break
                    elif column.get('name') not in {'id', 'twitid', 'date'}:
                        if new_label is None:
                            label_candidate = str(column.text).strip()
                            if label_candidate in {'0', '1', '-1'}:
                                new_label = 'negative' if label_candidate == '-1' else \
                                    ('positive' if label_candidate == '1' else 'neutral')
                                if new_text is not None:
                                    break
                if (new_text is None) or (new_label is None):
                    raise ValueError('File `{0}` contains some error!'.format(file_name))
                texts.append(new_text)
                labels.append(new_label)
            break
    return texts, labels

In [121]:
fXML_train = '/content/drive/My Drive/Colab Notebooks/bank_train_2016.xml'
fXML_test = '/content/drive/My Drive/Colab Notebooks/banks_test_etalon.xml'

In [122]:
texts, labels = load_sentirueval_2016(fXML)
texts_test, labels_test = load_sentirueval_2016(fXML_test)

In [123]:
print('Number of texts is {0}, number of labels is {1}.'.format(len(texts), len(labels)))

Number of texts is 9392, number of labels is 9392.




---

Рандомайзер и примеры

In [124]:
import random
for idx in random.choices(list(range(len(texts))), k=20):
    print('{0} => {1}'.format(labels[idx], texts[idx]))

neutral => завод ТМК реструктурировать долг кредит Сбербанк млрдamp  
neutral => банк Москва исключать слияние украинский  дочка   Вт Украина 
neutral => http  Сбербанк шаблон
neutral => газпромбанк руководство новосибирска http  газпромбанк  новосибирска
neutral => http брать кредит райффайзный банк
neutral => платить кредит сбербанк http 
neutral => Восточно сибирский сбербанк кредит http 
neutral => RT   Сбербанк ввести специальный режим работа офис подтопить ленинский район ЕАО
neutral => сбербанк кредит пенсионер процентный ставка http 
negative => Евросоюз наказывать Украина Сбербанк банк РФ
neutral => вт тольятть кредит http 
neutral => http вт потребительский кредит под залог недвижимость
negative =>  подача претензия просить приложить справка  сотрудник банк уверять  требоваться  основание отказать
neutral => вт тольятть кредит http 
neutral => http можно ли получить кредит пскова альфа банк
neutral => какой документ надо жилищный кредит сбербанк http 
negative =>  fyseoru  фи

In [125]:
positive_tweets = [texts[idx] for idx in range(len(texts)) if labels[idx] == 'positive']
for cur in positive_tweets[:5]: print(cur)
negative_tweets = [texts[idx] for idx in range(len(texts)) if labels[idx] == 'negative']
for cur in negative_tweets[:5]: print(cur)

  главный чтобы банк СБЕР ВТБ   
самый выгодный автокредит втб http 
http легко можно получить денежный кредит ы вт банк
снижение процентный ставка кредит недвижимость сбербанк http 
сбербанк прийти американец просить сфоткаться банкомат  D васто шок фотка полюбому получиться смешной xD
    обязательно про сбербанк писать  временами хлестко интерать магазин финт выкручивать 
   втб сбер точно  вопрос  блокировка операция запрет кредит 
Канада ввести санкция против Газпромбанк  ВЭБ  Новатэк российский оборонный компания
RT   Сбер ВТБ попасть под санкция  жоп
RT    Канада ввести санкция против оборонный сырьевой компания банк РФ  среди  частность    Газпромбанк     Внешэк …




---

Векторизация через Count

In [126]:
from nltk import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(lowercase=True, tokenizer=word_tokenize) # создаем экземпляр класса CV

In [127]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [128]:
vectorizer.fit(texts) # вызываем функцию fit() для обучения словам

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=<function word_tokenize at 0x7f258a5f9560>,
                vocabulary=None)

In [129]:
print(vectorizer.get_feature_names()[0:20])

['a', 'abncapital', 'aday', 'admitad', 'ag', 'agrob', 'ajhvf', 'alfa', 'alfabank', 'alfaclick', 'aliexpress', 'amp', 'android', 'api', 'app', 'apple', 'appsall', 'appstore', 'at', 'autofollow']


In [130]:
print(len(vectorizer.get_feature_names()))

7521


In [131]:
X = vectorizer.transform(texts) # вызываем функцию transform() для кодирования твитов как векторов

In [132]:
print(type(X)) # получаем матрицу для дальнейшего использования

<class 'scipy.sparse.csr.csr_matrix'>


In [133]:
print(texts[0])

http брать кредит тюмень альфа банк


In [134]:
print(X[0]) # (номер предложения, номер слова) кол-во слова в предложении (вектор слова?)

  (0, 172)	1
  (0, 650)	1
  (0, 800)	1
  (0, 982)	1
  (0, 2945)	1
  (0, 6715)	1


In [135]:
print(vectorizer.get_feature_names()[6321])

станция




---
Векторизация через tf-id

In [136]:
from sklearn.feature_extraction.text import TfidfTransformer # используем трансформер на матрице (после countvectorizer), vectorizer сам по себе 

In [137]:
transformer = TfidfTransformer().fit(X)

In [138]:
X_transformed = transformer.transform(X)

In [139]:
print(X_transformed[0]) # (предложение, номер слова) тф

  (0, 6715)	0.7555802916434852
  (0, 2945)	0.2149591146177843
  (0, 982)	0.3995836836356685
  (0, 800)	0.24308982435360857
  (0, 650)	0.3774147538391221
  (0, 172)	0.14727295262579873


In [140]:
print(X_transformed.shape) # (к-во предложений, к-во слов)

(9392, 7521)


In [141]:
print(vectorizer.get_feature_names()[7199])

часный


In [142]:
# получаем список таплов (слово, его idf)
tokens_with_IDF = list(zip(vectorizer.get_feature_names(), transformer.idf_))

In [143]:
for feature, idf in tokens_with_IDF[0:20]: print('{0:.6f} => {1}'.format(idf, feature))

9.049108 => a
9.454573 => abncapital
9.454573 => aday
9.454573 => admitad
9.049108 => ag
9.049108 => agrob
9.454573 => ajhvf
9.049108 => alfa
7.375131 => alfabank
8.761426 => alfaclick
9.454573 => aliexpress
6.484158 => amp
8.355961 => android
9.454573 => api
9.454573 => app
9.049108 => apple
9.454573 => appsall
9.454573 => appstore
8.068278 => at
8.201810 => autofollow


In [144]:
sorted_tokens_with_IDF = sorted(tokens_with_IDF, key=lambda it: (-it[1], it[0]))

In [145]:
for feature, idf in sorted_tokens_with_IDF[0:20]: print('{0:.6f} => {1}'.format(idf, feature))

9.454573 => abncapital
9.454573 => aday
9.454573 => admitad
9.454573 => ajhvf
9.454573 => aliexpress
9.454573 => api
9.454573 => app
9.454573 => appsall
9.454573 => appstore
9.454573 => autopaymts
9.454573 => back
9.454573 => bankvtbнадеяться
9.454573 => bash
9.454573 => basketballtime
9.454573 => best
9.454573 => bitcoin
9.454573 => bitcoiнлат
9.454573 => bitlydqeom
9.454573 => bitlydqxom
9.454573 => biznews




---

Разделяем тренировочную выборку на 80-20, занимаемся ее сортировкой

In [146]:
from sklearn.feature_selection import SelectPercentile, chi2

In [147]:
selector = SelectPercentile(chi2, percentile=20)

In [148]:
selector.fit(X_transformed, labels)

SelectPercentile(percentile=20, score_func=<function chi2 at 0x7f25895eb950>)

In [149]:
selected_tokens_with_IDF = [tokens_with_IDF[idx] for idx in selector.get_support(indices=True)]

In [150]:
print(len(selected_tokens_with_IDF))

1504


покажем 20 features (признаков) из 20% отобранных методом Хи-квадрат (по алфавиту)

In [151]:
for feature, idf in selected_tokens_with_IDF[0:20]: print('{0:.6f} => {1}'.format(idf, feature))

7.375131 => alfabank
9.049108 => apple
9.454573 => appstore
8.761426 => awards
8.538282 => boerse
9.049108 => brkng
9.049108 => cash
9.049108 => cemitsvetiс
9.049108 => cgi
9.454573 => come
8.355961 => d
9.454573 => diabloiii
9.049108 => dkk
9.454573 => ekb
9.454573 => euro
9.049108 => ff
7.012226 => finance
7.439670 => fitch
7.508663 => followback
9.049108 => followme


In [152]:
selected_and_sorted_tokens_with_IDF = sorted(selected_tokens_with_IDF, key=lambda it: (-it[1], it[0]))

покажем 20 features (признаков) из 20% отобранных методом Хи-квадрат (по idf и по алфавиту) 

In [153]:
for feature, idf in selected_and_sorted_tokens_with_IDF[0:20]: print('{0:.6f} => {1}'.format(idf, feature))

9.454573 => appstore
9.454573 => come
9.454573 => diabloiii
9.454573 => ekb
9.454573 => euro
9.454573 => gazetaru
9.454573 => hsbc
9.454573 => httpstcoiypfjojca
9.454573 => httpstcosutgdas
9.454573 => httpstcotyhbdxwx
9.454573 => iosандроид
9.454573 => london
9.454573 => lthttp
9.454573 => magaziнлат
9.454573 => mkru
9.454573 => moneynews
9.454573 => nalexandrowa
9.454573 => newzday
9.454573 => nskonline
9.454573 => picstitch


покажем 50 наименее важных features (признаков) из 20% отобранных методом Хи-квадрат (по idf и алфавиту)

In [154]:
for feature, idf in selected_and_sorted_tokens_with_IDF[-50:]: print('{0:.6f} => {1}'.format(idf, feature))

5.272523 => уралсиб
5.242445 => условие
5.206078 => банкомат
5.198960 => наличный
5.164113 => работа
5.117282 => миллиард
5.110767 => российский
5.091474 => автокредит
4.994428 => клиент
4.960334 => бизнес
4.938234 => ставка
4.911278 => новость
4.885030 => райффайзенбанк
4.864516 => калькулятор
4.815001 => деньги
4.810182 => ипотечный
4.791134 => ипотека
4.772442 => попасть
4.740548 => получить
4.700983 => оформить
4.688134 => …
4.650552 => список
4.646462 => рубль
4.622267 => ввести
4.606456 => заявка
4.571771 => против
4.527319 => новый
4.488238 => под
4.414379 => год
4.398327 => ес
4.339578 => сша
4.286934 => потребительский
4.247823 => онлайн
4.212826 => газпромбанк
4.210184 => »
4.189295 => «
3.708370 => брать
3.635975 => кредитный
3.619762 => сбербанок
3.595212 => санкция
3.502629 => альфа
3.496148 => вт
3.476954 => москва
3.329889 => карта
3.293366 => россельхозбанк
3.158385 => rt
3.144655 => втб
2.256015 => банк
1.994946 => кредит
1.366779 => http


имеет смысл, что ссылки, названия банков и финансовая лексика имеют меньшую значимость, поскольку они часто поовторяются



---

